# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tianfu,34.2650,108.5904,274.68,43,5,2.91,CN,"time.struct_time(tm_year=2025, tm_mon=2, tm_md..."
1,1,kingston,17.9712,-76.7928,300.05,78,75,6.69,JM,"time.struct_time(tm_year=2025, tm_mon=2, tm_md..."
2,2,olonkinbyen,70.9224,-8.7160,271.93,83,100,8.73,SJ,"time.struct_time(tm_year=2025, tm_mon=2, tm_md..."
3,3,invercargill,-46.4118,168.3471,281.49,94,38,1.64,NZ,"time.struct_time(tm_year=2025, tm_mon=2, tm_md..."
4,4,vilyuchinsk,52.9318,158.4054,265.32,93,96,1.98,RU,"time.struct_time(tm_year=2025, tm_mon=2, tm_md..."


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
mapplot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    color = 'City',
    size = 'Humidity'
)
# Display the map
# YOUR CODE HERE
mapplot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Celsius feels more natural than kelvin so this is just for me
# since it's simple
city_data_df['tempc'] = city_data_df['Max Temp'] - 273.15

# Narrow down cities that fit criteria and drop any results with null values
idealdf = city_data_df[(city_data_df['tempc'] <= 15) & (city_data_df['tempc'] >= (-6))]
idealdf2 = idealdf[~(idealdf['Cloudiness'] < 60)]
# Cold and cloudy 

# Display sample data
idealdf2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,tempc
2,2,olonkinbyen,70.9224,-8.7160,271.93,83,100,8.73,SJ,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",-1.22
7,7,port-aux-francais,-49.3498,70.2200,278.92,99,100,9.49,TF,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",5.77
8,8,gazojak,41.1892,61.4060,276.18,46,81,2.68,TM,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",3.03
13,13,albany,42.6512,-73.7550,275.14,63,75,11.32,US,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",1.99
15,15,bethel,60.7922,-161.7558,272.34,100,100,6.69,US,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",-0.81
...,...,...,...,...,...,...,...,...,...,...,...
550,550,al shamkhah city,35.4425,-95.9864,278.08,69,89,4.48,US,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",4.93
564,564,bayevo,53.2686,80.7792,267.43,96,100,1.70,RU,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",-5.72
568,568,billund,55.7247,9.1196,277.19,93,100,4.63,DK,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",4.04
570,570,yarmouth,52.6072,1.7315,282.01,81,97,8.29,GB,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",8.86


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealdf2.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,olonkinbyen,SJ,70.9224,-8.7160,83,None
7,port-aux-francais,TF,-49.3498,70.2200,99,None
8,gazojak,TM,41.1892,61.4060,46,None
13,albany,US,42.6512,-73.7550,63,None
15,bethel,US,60.7922,-161.7558,100,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey':geoapify_key,
    'categories':'accommodation.hotel',
    'limit':1
}

# Print a message to follow up the hotel search
print("Starting hotel search")
i = 0 # Using row was throwing a lot of errors so i used my own

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = hotel_df.iloc[i, 3]
    lat = hotel_df.iloc[i, 2]
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    i+=1

# Display sample data
hotel_df

Starting hotel search
olonkinbyen - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
gazojak - nearest hotel: No hotel found
albany - nearest hotel: Renaissance Albany Hotel
bethel - nearest hotel: Bentley's Porter House B & B
uddevalla - nearest hotel: Carlia
north bend - nearest hotel: Salish Lodge & Spa
vradiyivka - nearest hotel: No hotel found
keflavik - nearest hotel: B&B Guesthouse
sandnessjoen - nearest hotel: Scandic Syv Søstre
puerto natales - nearest hotel: Los Pinos
college - nearest hotel: Hyatt Place Ottawa West
hamilton - nearest hotel: Homewood Suites by Hilton
asau - nearest hotel: Hotel Romanița
island harbour - nearest hotel: No hotel found
buchanan - nearest hotel: Holiday Inn Express & Suites Peekskill-Lower Hudson Valley
punta arenas - nearest hotel: Hotel Cabo de Hornos
port saint john's - nearest hotel: Hotel Cabo de Hornos
springfield - nearest hotel: Wyndham Springfield City Centre
milford haven - nearest hotel: Tŷ Milford Waterfront
gu

,City,Country,Lat,Lng,Humidity,Hotel Name
2,olonkinbyen,SJ,70.9224,-8.7160,83,No hotel found
7,port-aux-francais,TF,-49.3498,70.2200,99,Keravel
8,gazojak,TM,41.1892,61.4060,46,No hotel found
13,albany,US,42.6512,-73.7550,63,Renaissance Albany Hotel
15,bethel,US,60.7922,-161.7558,100,Bentley's Porter House B & B
...,...,...,...,...,...,...
550,al shamkhah city,US,35.4425,-95.9864,69,No hotel found
564,bayevo,RU,53.2686,80.7792,96,"Гостиница ""Кулунда"""
568,billund,DK,55.7247,9.1196,93,Refborg Hotel Billund
570,yarmouth,GB,52.6072,1.7315,81,New Beach Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [37]:
%%capture --no-display

# Configure the map plot
mapplot2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    color = 'City',
    size = 'Humidity',
    hover_cols = ['Hotel Name', 'Country']
)
# Display the map
# YOUR CODE HERE
mapplot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)